In [20]:
pip install yfinance


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install json

In [ ]:
pip install pandas

In [ ]:
pip install requests

In [ ]:
pip install openpyxl

In [1]:
import yfinance as yf
import json
import pandas as pd
import requests
from openpyxl import load_workbook

### loading the data, if file name is updated just change `file_path`

In [2]:
file_path = "/Users/counterpartventures/Desktop/Data/Survey_Data_V5.xlsx"
df = pd.read_excel(file_path, sheet_name="tickers")

### defining functions to get metrics

In [3]:
def get_market_cap(ticker):
    company = yf.Ticker(ticker)
    return company.info.get("marketCap", None)

def get_currency(ticker):
    company = yf.Ticker(ticker)
    return company.info.get("currency", None)

def get_revenue(ticker):
    company = yf.Ticker(ticker)
    revenue = company.financials.loc['Total Revenue'].dropna()
    return revenue.iloc[0]

### Getting metrics from Yahoo Finance, adding them to a new df `results`

In [4]:
market_caps = []
revenues = []
currencies = []

# pulling data

for ticker in df["ticker_symbol"]:
    market_cap = get_market_cap(ticker)
    revenue = get_revenue(ticker)
    currency = get_currency(ticker)

    market_caps.append(market_cap)
    revenues.append(revenue)
    currencies.append(currency)

In [ ]:
# creating new df with data pull

result = pd.DataFrame({
    'company': df['corp_parent'],
    'market_cap': market_caps,
    'revenue': revenues,
    'currency' : currencies
})

### Converting Currency to USD

In [5]:
# Conversion rate API

api_key = '225a1d628c6d8905e45fa173'
url = f'https://v6.exchangerate-api.com/v6/{api_key}/latest/USD'
response = requests.get(url)
conversions = response.json()

result['currency'] = result['currency'].str.upper()

# function to covert to USD

def convert_usd(amt, currency):
    if currency == 'USD' or pd.isna(amt):
        return amt
    elif amt is not None:
        if currency in conversions['conversion_rates']:
            rate = conversions['conversion_rates'][currency]
            return amt / rate
    return None

# converting market cap and revenue to $B

result['market_cap_usd'] = result.apply(
    lambda row: row['market_cap'] if row['currency'] == 'USD'
    else convert_usd(row['market_cap'], row['currency']),
    axis = 1
) / 1e9

result['revenue_usd'] = result.apply(
    lambda row: row['revenue'] if row['currency'] == 'USD'
    else convert_usd(row['revenue'], row['currency']),
    axis = 1
) / 1e9

In [6]:
## Converting original values to billions

result['market_cap'] = (result['market_cap'] / 1e9).astype(str)
result['revenue'] = (result['revenue'] / 1e9).astype(str)

result['og_market_cap'] = result[['currency', 'market_cap']].agg(' '.join, axis=1)
result['og_revenue'] = result[['currency', 'revenue']].agg(' '.join, axis=1)

# selecting variables

result = result[['company', 'og_market_cap', 'market_cap_usd', 'og_revenue', 'revenue_usd']]
result = pd.DataFrame(result)

### Adding `result` to the master excel

In [7]:
new_sheet = 'market_cap_data'

In [8]:
with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    result.to_excel(writer, sheet_name=new_sheet, index=False)